### Part B


#### Task 2


In [ ]:
from json import loads
from kafka3 import KafkaConsumer
import os
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statistics
from datetime import datetime
from pymongo import MongoClient
import time

In [ ]:
!pip install folium

In [ ]:
import folium

In [ ]:
# change directory (local machine only)
# os.chdir("A2")
# os.chdir("data")

# dirs = os.listdir(os.getcwd())
# print(dirs)

In [ ]:
hostip = "192.168.10.125"

In [ ]:
%matplotlib notebook

In [ ]:
# reference code taken from FIT3182 applied session week 10
def connect_kafka_consumer():
    try:
        return KafkaConsumer(
            "climate",
            bootstrap_servers=[f"{hostip}:9092"],
            auto_offset_reset="earliest",
            value_deserializer=lambda x: json.loads(x.decode("utf-8")),
            api_version=(0, 10),
        )
    except Exception as ex:
        print("Exception while connecting to Kafka:", str(ex))


def init_plots():
    try:
        # create a new figure
        width = 10
        height = 5

        # set size
        fig = plt.figure(figsize=(width, height))

        # add a subplot with no frame
        fig.suptitle("Air Temperature vs Arrival Time")

        # create a subplot
        ax = fig.add_subplot(221)

        # set x and y axis labels
        ax.set_xlabel("Arrival Time")
        ax.set_ylabel("Air Temperature (Celsius)")

        # set limits
        ax.set_ylim(0, 50)

        # set ticks
        ax.set_yticks([0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

        # show and draw the plot
        fig.show()
        fig.canvas.draw()
        return fig, ax
    except Exception as ex:
        print(str(ex))


def annotate_max(x, y, ax=None):
    # if no axis object is passed, get the current axis
    if not ax:
        ax = plt.gca()

    # get the maximum value of y
    ymax = max(y)

    # get the x value of the maximum value of y
    xpos = y.index(ymax)

    # get the x value of the maximum value of y
    xmax = x[xpos]

    # format the time
    formatted_time = (
        datetime.strptime(xmax, "%d/%m/%y %H:%M:%S").strftime("%d/%m/%y")
        if xmax.endswith("00:00:00")
        else xmax
    )
    ax.annotate(
        "Max: Date: {}, Temperature: {} C".format(formatted_time, ymax),
        xy=(xmax, ymax),
        xytext=(xmax, ymax + 5),
        textcoords="offset points",
        ha="center",
        va="bottom",
        arrowprops=dict(facecolor="red", shrink=0.05),
    )


def annotate_min(x, y, ax=None):
    if not ax:
        ax = plt.gca()
    ymin = min(y)
    xpos = y.index(ymin)
    xmin = x[xpos]
    formatted_time = (
        datetime.strptime(xmin, "%d/%m/%y %H:%M:%S").strftime("%d/%m/%y")
        if xmin.endswith("00:00:00")
        else xmin
    )
    ax.annotate(
        "Min: Date: {}, Temperature: {} C".format(formatted_time, ymin),
        xy=(xmin, ymin),
        xytext=(xmin, ymin - 5),
        textcoords="offset points",
        ha="center",
        va="top",
        arrowprops=dict(facecolor="orange", shrink=0.05),
    )


def consume_messages(consumer, fig, ax):
    x, y = [], []
    try:
        for message in consumer:
            # get the message value and append it to the y list
            data = message.value

            # convert the date to a datetime object
            message_date = datetime.fromisoformat(data["created_date"])

            # format the date
            clean_date = message_date.strftime("%d/%m/%y")

            # append the date and temperature to the x and y lists
            x.append(clean_date)

            # append the temperature to the y list
            y.append(data["air_temperature_celcius"])

            # start when 10 data points have been collected
            if len(y) > 10:

                # clear the axis
                ax.clear()

                # plot the data
                ax.plot(x, y)
                ax.set_xlabel("Arrival Time")
                ax.set_ylabel("Air Temperature (Celsius)")
                ax.set_ylim(0, 50)
                ax.set_yticks([0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50])
                ax.tick_params(axis="x", labelrotation=45)

                plt.xticks(rotation=45, ha="right")

                # annotate the max and min values
                annotate_max(x, y, ax)
                annotate_min(x, y, ax)

                # draw the plot
                fig.canvas.draw()

                # remove the first value from the x and y lists
                x.pop(0)
                y.pop(0)
        plt.close("all")
    except Exception as ex:
        print("Error in consuming messages:", str(ex))


if __name__ == "__main__":
    consumer = connect_kafka_consumer()
    fig, ax = init_plots()
    consume_messages(consumer, fig, ax)

In [ ]:
client = MongoClient(hostip, 27017)
db = client["fit3182_assignment_db"]
collection = db["processed_data"]

# get the number of fire records by hour
query = [
    {"$unwind": "$hotspots"},
    {
        "$addFields": {
            "full_datetime": {
                "$dateFromString": {
                    "dateString": {
                        "$concat": [
                            {"$substr": ["$date", 0, 10]},
                            "T",
                            "$hotspots.datetime",
                        ]
                    }
                }
            }
        }
    },
    {"$group": {"_id": {"$hour": "$full_datetime"}, "count": {"$sum": 1}}},
    {"$sort": {"_id": 1}},
]

# execute the query
result = list(collection.aggregate(query))

# get the hours and fire counts
hours = [doc["_id"] for doc in result]

# get the fire counts
fire_counts = [doc["count"] for doc in result]

# plot the data 0 to 24
all_hours = range(24)
all_counts = [0] * 24

# update the counts
for hour, count in zip(hours, fire_counts):
    all_counts[hour] = count

# plot the data
plt.figure(figsize=(10, 6))
bars = plt.bar(all_hours, all_counts, color="b")
plt.xlabel("Hour of Day")
plt.ylabel("Number of Fire Records")
plt.title("Number of Fire Records by Hour")
plt.xticks(all_hours, [f"{h:02d}:00" for h in all_hours], rotation=45)
# plt.yticks(range(0, 200, 50))
plt.tight_layout()

# add the count values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        yval,
        f"{int(yval)}",
        ha="center",
        va="bottom",
        fontsize=9,
        color="black",
    )

plt.show()

In [ ]:
client = MongoClient(hostip, 27017)
db = client["fit3182_assignment_db"]
collection = db["processed_data"]

# start on centre of victoria state
fire_map = folium.Map(location=[36.8233, 144.2944], zoom_start=3)

# iterate through the collection
for doc in collection.find():
    for hotspot in doc["hotspots"]:
        # set all variables to be used
        latitude = hotspot["latitude"]
        longitude = hotspot["longitude"]
        cause = doc["climate"]["cause"]
        air_temperature = doc["climate"]["air_temperature_celcius"]
        surface_temperature = hotspot["surface_temperature_celcius"]
        relative_humidity = doc["climate"]["relative_humidity"]
        confidence = hotspot["confidence"]

        # add the popup text
        popup_text = (
            f"Air Temperature: {air_temperature}<br>"
            f"Surface Temperature: {surface_temperature}<br>"
            f"Relative Humidity: {relative_humidity}<br>"
            f"Confidence: {confidence}"
        )

        # set the color based on the cause
        if cause == "natural":
            color = "blue"
        else:
            color = "red"

        # add the marker to the map based on color and cause
        folium.Marker(
            location=[latitude, longitude],
            popup=folium.Popup(popup_text, max_width=300),
            icon=folium.Icon(color=color),
        ).add_to(fire_map)

# save the map
fire_map.save("fire_locations_map.html")

In [ ]:
fire_map